<a href="https://colab.research.google.com/github/learnvisionAI/ObjectDetection_YOLO/blob/main/Session2_YOLOV8_Obj_Detection_CustomDataset/Potholes_Detection__Step_by_Step_base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Importing the Required Libraries**

In [ ]:
import os
import glob
from IPython.display import Image, display
from IPython import display

# **In the First Step, We need to check whether we have access to the GPU or not**

In [ ]:
!nvidia-smi

In [ ]:
HOME = os.getcwd()

In [ ]:
print(HOME)

#**Installing Ultralytics using Pip Install**

In [ ]:
!pip install ultralytics==8.0.0

##Checking whether YOLOv8 is Installed and its working Fine

In [ ]:
import ultralytics

In [ ]:
ultralytics.checks()

#**Importing the Potholes Images Dataset from Roboflow**

##### https://universe.roboflow.com/devashish-workspace/pothole-detection-2-28w0d/dataset/2/images/?split=train&numImages=60

In [ ]:
!mkdir {HOME}/datasets


In [ ]:
!pwd

In [ ]:
%cd {HOME}/datasets


In [ ]:
!pwd

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="16XuuBsC5XqwYkDJ8HdL")
project = rf.workspace("vidhya-shankar-ely0e").project("potholedetectionyolov8-k6l02")
dataset = project.version(2).download("yolov5")

#**Train the YOLOv8 Model on the Custom Dataset**

In [ ]:
%cd {HOME}

In [ ]:
%cd {dataset.location}

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8s.pt data=/content/datasets/potholeDetectionYoloV8-2/data.yaml epochs=70 imgsz=640

In [ ]:
!ls {HOME}//runs/detect/train3

#**Displaying the Confusion Matrix**

In [ ]:
#Confusion matrix is the chart that shows how our model handles different classes
#68% of the time the model detected correctly that there is a pothole, while 32% of the time, when the pothole is there but the model is not
#able to detect it
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train3/confusion_matrix.png', width=900)

#**Training and Validation Loss**

In [ ]:
# Here is the graph of the training and validation loss
#box loss and class loss is important
# The behavior of the model is convincing the model is coverging, Training more will give better results
%cd {HOME}

Image(filename=f'{HOME}/runs/detect/train3/results.png', width=600)

In [ ]:
#Model Prediction on validation batch. These image are not used strictly for training so it is always better to take a
#look and see how model is behaving
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train3/val_batch0_pred.jpg', width=800)

#**Validate Custom Model**

In [ ]:
#Here, we are taking the model best weights and using them to validate the model. Similarly as before we are using
#CLI to do that, The only difference is our mode = val instead of train
#Validation Script is using test dataset that was not used beforew

%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train3/weights/best.pt data=/content/datasets/potholeDetectionYoloV8-2/data.yaml

#**Inference with Custom Model**

In [ ]:
# Testing the Model on Test Dataset images
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train3/weights/best.pt conf=0.25 source=/content/datasets/potholeDetectionYoloV8-2/test/images

In [ ]:
Image("/content/runs/detect/predict2/16_jpg.rf.0b3dbe349c7e0796a6dea8712d6474b5.jpg")

#**Testing on a Demo Video**

In [ ]:
!gdown 'https://drive.google.com/uc?id=191krl3cz61oZCWwNyd5TLTSdUIb5HRI9&confirm=t'

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train3/weights/best.pt conf=0.25 source='/content/demo.mp4'

#**Display the Demo Video**

In [ ]:
!rm "/content/result_compressed.mp4"

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = '/content/runs/detect/predict4/demo.mp4'

# Compressed video path
compressed_path = "/content/result_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)